# WebNLG Data-To-Text Predictions

#### Download the WebNLG dataset

In [ ]:
!git clone https://gitlab.com/shimorina/webnlg-dataset.git

### Setup WebNLG

#### Imports

In [ ]:
!pip3 install pandas
!pip3 install openai
!pip3 install python-dotenv             
!pip3 install typing

In [ ]:
import pandas as pd
from dotenv import load_dotenv
from Import import create_webnlg_df, create_dataframe
from Requests import *
from Export import *
from Prompts import *
from Models import *

load_dotenv()

#### Create Dataframe from WebNLG dataset

Add first 10 values to dataframe

In [ ]:
file_path = "./webnlg-dataset/release_v3.0/en/json/dev/v3.0_dev_set.json"
data_set_size = 1667
sample_size = 20

webnlg_df = create_dataframe(file_path, data_set_size, sample_size, random_sample=True)
webnlg_df.head()

### Call LLM APIs

In [ ]:
model: ModelType = ModelType.GPT3

print(model.value)

#### OpenAI Chat models

In [ ]:
model = ModelType.GPT3

In [ ]:
response = send_to_openai_chat([{"role": "user", "content": 'Hi, how are you doing'}], model.value)
print(response)

In [ ]:
def convert_triple_to_text_openai_chat(model: str, input_tripleset, print_prompt_template: bool = False) -> Tuple[str, float]:
    prompt_template = get_few_shot_chat_prompt(input_tripleset, True)
    
    if print_prompt_template:
        print(f"Prompt template: {prompt_template}")
    
    return send_to_openai_chat(prompt_template, model)

input = webnlg_df["modifiedtripleset"][2]
result = convert_triple_to_text_openai_chat(model.value, input, True)
print(f"""
Output: {result[0]}
Execution time: {result[1]}
""")

Fill the dataframe with OpenAi gpt3 predictions

In [ ]:
def add_gpt3_predictions_to_df(model: str, dataframe: pd.DataFrame, input_column: str) -> pd.DataFrame:
    # Get the predictions for all entries of the input_column in the dataframe
    response = [convert_triple_to_text_openai_chat(model, x) for x in dataframe[input_column]]

    predictions = [x[0] for x in response]
    execution_time = [x[1] for x in response]

    dataframe[f"prediction_{model}"] = predictions
    dataframe[f"execution_time_{model}"] = execution_time
    
    return dataframe

webnlg_df = add_gpt3_predictions_to_df(model.value, webnlg_df, "modifiedtripleset")
webnlg_df.head()

#### OpenAI Completion Models

In [ ]:
model = ModelType.DAVINCI

In [ ]:
def convert_triple_to_text_openai_completion(model: ModelType, input_tripleset, print_prompt_template: bool = False) -> Tuple[str, float]:
    prompt_template = get_zero_shot_completion_prompt(input_tripleset)

    if print_prompt_template:
        print(f"Prompt template: {prompt_template}")
    
    return send_to_openai_completion(prompt_template, model.value)

input = webnlg_df["modifiedtripleset"][2]
result = convert_triple_to_text_openai_completion(model, input, True)
print(f"""
Output: {result[0]}
Execution time: {result[1]}s
""")

Fill the dataframe with OpenAi text-davinci-003 predictions

In [ ]:
def add_davinci_predictions_to_df(model: ModelType, dataframe: pd.DataFrame, input_column: str) -> pd.DataFrame:
    # Get the predictions for all entries of the input_column in the dataframe
    response = [convert_triple_to_text_openai_completion(model, x) for x in dataframe[input_column]]

    predictions = [x[0] for x in response]
    execution_time = [x[1] for x in response]

    dataframe[f"prediction_{model.value}"] = predictions
    dataframe[f"execution_time_{model.value}"] = execution_time

    return dataframe

webnlg_train_df = add_davinci_predictions_to_df(model, webnlg_df, "modifiedtripleset")
webnlg_train_df.head()

#### Local Server (Vicuna / LLaMA)

The models that run on the local server with FastChat (i.e. LLaMA, Vicuna), do not need to send a system message to the server in the api call. For these models the system message is already added as part of FastChat conversation template (it can be changed in the file fastchat/conversation.py).  
To verify that the correct system message is used, you can make the following request:

In [ ]:
!curl --location --request POST 'http://0.0.0.0:21002/worker_get_conv_template'

In [ ]:
# "VICUNA" or "LLAMA"
model = ModelType.LLAMALORA

In [ ]:
result = send_to_local_server_chat([{"role": "user", "content": 'What have I asked you before?'}], model.value)
print(f"Answer: {result[0]}\nExecution time: {result[1]}s")

In [ ]:
def convert_triple_to_text_local_server_chat(model: ModelType, input_tripleset, print_prompt_template: bool = False) -> Tuple[str, float]:
    prompt_template = get_few_shot_chat_prompt(input_tripleset)
    
    if print_prompt_template:
        print(f"Prompt template: {prompt_template}")

    return send_to_local_server_chat(prompt_template, model.value)
    

input = webnlg_df["modifiedtripleset"][0]
result = convert_triple_to_text_local_server_chat(model, input, True)
print(f"""
Output: {result[0]}
Execution time: {result[1]}
""")

You can also use the completion endpoint instead

In [ ]:
def convert_triple_to_text_local_server_completion(model: ModelType, input_tripleset, print_prompt_template: bool = False) -> Tuple[str, float]:
    prompt_template = get_few_shot_completion_prompt(input_tripleset)
    
    if print_prompt_template:
        print(f"Prompt template: {prompt_template}")

    return send_to_local_server_completion(prompt_template, model.value)
    

input = webnlg_train_df["modifiedtripleset"][2]
result = convert_triple_to_text_local_server_completion(model, input, True)
print(f"""
Output: {result[0]}
Execution time: {result[1]}s
""")

Fill the dataframe with Vicuna predictions

In [ ]:
def add_local_model_predictions_to_df(model: ModelType, dataframe: pd.DataFrame, input_column: str) -> pd.DataFrame:
    # Get the predictions for all entries of the input_column in the dataframe
    response = [convert_triple_to_text_local_server_chat(model, x, True) for x in dataframe[input_column]]
    predictions = [x[0] for x in response]
    execution_time = [x[1] for x in response]

    dataframe[f"prediction_{model}"] = predictions
    dataframe[f"execution_time_{model}"] = execution_time

    return dataframe    

webnlg_df = add_local_model_predictions_to_df(model, webnlg_df, "modifiedtripleset")
webnlg_df.head()

#### Finetuned Model (LLaMA-Lora)

In [ ]:
model = ModelType.LLAMALORA

print(model.value)

In [ ]:
result = send_to_local_gradio_server(get_finetune_instruction(), get_fintune_input(webnlg_train_df["modifiedtripleset"][2]))

In [ ]:
result = json.loads(result)
print(result)

In [ ]:
print(f"Answer: {result['data']}\nExecution time: {result['duration']}s")

In [ ]:
input = webnlg_train_df["modifiedtripleset"][2]
result = convert_triple_to_text_gradio_server(lambda: get_finetune_instruction(), get_fintune_input(input))
print(f"""
Input: {input}
Output: {result[0]}
Execution time: {result[1]}s
""")

Fill the dataframe with predictions of local model

In [ ]:
def add_finetuned_model_predictions_to_df(model: ModelType, dataframe: pd.DataFrame, input_column: str) -> pd.DataFrame:
    # Get the predictions for all entries of the input_column in the dataframe
    response = [convert_triple_to_text_gradio_server(lambda: get_finetune_instruction(), get_fintune_input(input)) for input in dataframe[input_column]]
    predictions = [x[0] for x in response]
    execution_time = [x[1] for x in response]

    dataframe[f"prediction_{model.value}"] = predictions
    dataframe[f"execution_time_{model.value}"] = execution_time
    
    return dataframe

webnlg_train_df = add_finetuned_model_predictions_to_df(model, webnlg_train_df, "modifiedtripleset")
webnlg_train_df.head()

#### Copy-Baseline

As a baseline, we simply copy "object", "property" and "subject" to the output. We separate each triple with a "."

In [ ]:
model = ModelType.BASELINE

In [ ]:
def create_copy_baseline_string_from_triples(triples: list) -> str:
    result = ""

    for triple in triples: 
        result += f"{triple['object']} {triple['property']} {triple['subject']}. "
    
    return result

create_copy_baseline_string_from_triples(webnlg_train_df["modifiedtripleset"][2])

In [ ]:
def add_copy_baseline_to_df(model: ModelType, dataframe: pd.DataFrame, input_column: str) -> pd.DataFrame:
    dataframe[f"prediction_{model.value}"] = [create_copy_baseline_string_from_triples(x) for x in dataframe[input_column]]
    dataframe[f"execution_time_{model.value}"] = 0.0
    
    return dataframe

webnlg_train_df = add_copy_baseline_to_df(model, webnlg_train_df, "modifiedtripleset")
webnlg_train_df.head()

### Export to Excel

#### Export webnlg_train_df to csv

In [ ]:
export_dataframe_to_csv(webnlg_df, "Results/csv/Predictions/Dev", f"{model.value}_few-shot-chat")

## Automatic Predictions

In [ ]:
def append_to_dataframe_and_export(dataframe: pd.DataFrame, index: int, prediction: str, execution_time: float, model: ModelType, prompt_type: str, output_path: str):
    # add prediction and execution time to the dataframe columns at index
    dataframe.loc[index, f"prediction_{model.value}"] = prediction
    dataframe.loc[index, f"execution_time_{model.value}"] = execution_time    

    # append the dataframe row at index to the csv file
    header = index == 0
    dataframe.iloc[[index]].to_csv(f"{output_path}/{model.value}_{prompt_type}.csv", mode='a', header=header, index=False)
    print(f"Exported prediction for example with id: {dataframe.loc[index, 'id']}")

# append_to_dataframe_and_export(webnlg_df, 0, "This is the test prediction 0", 0.1, ModelType.LLAMALORA, "test-prompt", "Results/csv/Predictions/Train")

In [ ]:
def handle_llama_lora(dataframe: pd.DataFrame, input_column: str, sample_size: int, start_index: int, output_path: str) -> pd.DataFrame:
    inputs = dataframe[input_column]

    for index, input in enumerate(inputs):
        if (index < start_index):
            continue

        if (index == (start_index + sample_size)):
            break

        response = convert_triple_to_text_gradio_server(lambda: get_finetune_instruction(), get_fintune_input(input))

        prediction = response[0]
        execution_time = response[1]

        append_to_dataframe_and_export(dataframe, index, prediction, execution_time, ModelType.LLAMALORA, "finetuned", output_path)

    return dataframe
        

def create_predictions_for_dataframe(model: ModelType, dataset_path: str, dataset_size: int, sample_size: int, prompt_generator: Callable[[str], str], prompt_type: str, output_path: str, start_id: int = 1) -> pd.DataFrame:
    '''start_id is the id of the first example in the dataset that should be used for prediction (assuming the dataset is sorted by id ascending)'''
    start_index = start_id - 1
    max_tokens = 128
    input_column = "modifiedtripleset"
    prompt_templates: List[List[dict]]
    dataframe = create_webnlg_df(dataset_path, dataset_size)

    if (model is ModelType.LLAMALORA):
        return handle_llama_lora(dataframe, input_column, sample_size, start_index, output_path)

    prompt_templates = [prompt_generator(x) for x in dataframe[input_column]]
    print(prompt_templates[start_index])

    for index, prompt_template in enumerate(prompt_templates):
        start_time = time.time()
        if (index < start_index):
            continue

        if (index == (start_index + sample_size)):
            break

        if (model is ModelType.LLAMA or model is ModelType.VICUNA or model is ModelType.LORA):
            # if you want to test the completion endpoints of LLaMA or Vicuna you can change the line below to:
            # response = send_to_local_server_completion(prompt_template, model.value)
            response = send_to_local_server_chat(prompt_template, model.value, max_tokens=max_tokens)
        elif (model is ModelType.GPT3):
            response = send_to_openai_chat(prompt_template, model.value, max_tokens=max_tokens)
        elif (model is ModelType.DAVINCI):
            response = send_to_openai_completion(prompt_template, model.value, max_tokens=max_tokens)

        prediction = response[0]
        execution_time = response[1]

        append_to_dataframe_and_export(dataframe, index, prediction, execution_time, model, prompt_type, output_path)
        print(time.time() - start_time)
    return dataframe
    
    
predictions_df = create_predictions_for_dataframe(ModelType.LORA, "./webnlg-dataset/release_v3.0/en/json/test/v3.0_test_set.json", 1779, 1779, lambda x: get_few_shot_chat_prompt(x, False), "few-shot-chat", "Results/csv/Predictions/Test", 1)

In [ ]:
predictions_df.head(10)